# Task No3 Indexes

## 0. Task

1. Read Lectures #5 Indexes
2. Upload ONLY 80% dataset from Task1 to the created database from Task2.
3. Create 3 queries to this database based on functionality from Task2.
4. Implement queries in SQL.
5. Run the queries and collect the query statistics (execution time and number of reads) and execution plan.
6. Create B-tree indexes to improve these queries.
7. Run the queries again and collect the query statistics (execution time and number of reads) after adding indexes and execution plan. Be sure that query plans contain created indexes.
8. Compare execution time and query statistics without and with indexes.
9. Make a conclusion.


#### Student: Jingyu Yan


## 1. Example

Translate to English: "According to Task NO.2, we have established an e-commerce system database for a cosmetics brand, and imported 80% of the data from the dataset into this database for the preparation of experimental data.

In [9]:
from setting import get_engine
from sqlalchemy import MetaData
import pandas as pd
import time
from functools import wraps

# Create the engine
engine = get_engine()

def cost(tag=''):
    """
    A decorator for timing a function execution.
    """
    def wrapper(fn):
        @wraps(fn)
        def wrapper_use_time(*args, **kw):
            t1 = time.time()
            try:
                res = fn(*args, **kw)
            except Exception as e:
                print(f"Error in {fn.__name__}({tag}): {str(e)}")
                return None
            else:
                t2 = time.time()
                print(f"{tag}@Cost of {fn.__name__}(): {round((t2 - t1) * 1000, 3)} ms")
                return res
        return wrapper_use_time
    return wrapper

Display database information

In [10]:
def display_information(engine):
    metadata = MetaData()
    metadata.reflect(engine)

    data = []
    for table_name in metadata.tables:
        table = metadata.tables[table_name]
        columns = table.columns.keys()
        data_types = [str(column.type) for column in table.columns.values()]
        count = engine.execute(f"SELECT COUNT(*) FROM {table_name}").scalar()
        data.append({"Table Name": table_name, "Columns": ", ".join([item for item in columns]), "Number of Rows": count})

    df = pd.DataFrame(data)
    df.style.set_table_styles(
        [{'selector': 'th', 'props': [('font-size', '12pt'), ('text-align', 'center')]}]
    ).set_properties(**{'background-color': 'lavender', 'color': 'black', 'border-color': 'black'})
    
    return df

display_information(engine)

,Table Name,Columns,Number of Rows
0,cart_items,"id, cart_id, product_id, quantity, added_time",33155
1,carts,"id, user_id",10032
2,users,"id, username, email, password_hash, registrati...",10032
3,products,"id, name, category_id, brand, price, descripti...",1472
4,categories,"id, name, code, parent_category_id",6
5,events,"id, user_id, product_id, event_type, event_time",2286930
6,order_details,"id, order_id, product_id, quantity, price, dis...",145271
7,orders,"id, user_id, order_date, status, total_price",48413


The 'events' table, being the one with the most data and the most frequent interactions between the system and users, should be the primary focus of our example this time.

### Implement queries in SQL

I have chosen to perform a query on the 'events' table to conduct an experiment on database indexing.

In [11]:
@cost("SELECT")
def query_events_by_date_range(engine, start_date, end_date):
    """
    Query events in the given date range.
    """
    query = f"""
        SELECT id, 
            user_id, 
            event_type, 
            event_time 
        FROM events
        WHERE event_time >= '{start_date}'
          AND event_time < '{end_date}';
    """
    with engine.connect() as connection:
        result = connection.execute(query, {'start_date': start_date, 'end_date': end_date})
    
    return result

@cost("INDEX")
def create_index_by_events_date():
    query = """
        CREATE INDEX idx_event_time ON events(event_time);
    """
    with engine.connect() as connection:
        connection.execute(query)

### Executing the first query before using an index.

Run the queries and collect the query statistics (execution time and
number of reads) and execution plan.

In [14]:
res = query_events_by_date_range(engine, "2017-06-22 00:00:00", "2017-06-23 00:00:00")

SELECT@Cost of query_events_by_date_range(): 47.374 ms


This query took **437.507ms** ms to execute.

*Note: MySQL may not directly provide detailed query performance statistics such as "Scan count", "logical reads", or "physical reads" like SQL Server does, which is unfortunate considering my computer runs a Unix-like system that may not support these tools. Other tools might offer this functionality, but I haven't found them yet.*

### Create B-tree indexes to improve these queries.

For this query task, we establish a B-tree indexes index.

In [5]:
# Create B-tree index
create_index_by_events_date()

INDEX@Cost of create_index_by_events_date(): 4392.725 ms


It's important to note that creating an index can be time-consuming. This is due to the need for the database to scan and sort the relevant column's data, construct the index structure like a B-tree, and write this new index to disk. The process is influenced by the size of the data and the available system resources.

### Query again after creating an index

When I have created the index, I will execute the same query task as before.

In [8]:
res = query_events_by_date_range(engine, "2017-06-22 00:00:00", "2017-06-23 00:00:00")

SELECT@Cost of query_events_by_date_range(): 28.068 ms


It is easily observed that the time taken to execute the query has been reduced.Executing the query locally took 9.677ms.

## 2. Compare

Comparing queries before and after creating an index, it's evident that in a table with over **900,000** entries, a conditional search using an index is significantly faster than one without an index.

## 3. Make a Conclusion

- Significant Speed Improvement: Queries with indexing in a table of over 900,000 records show a drastic speed increase, from over 400ms to just around 9.677ms.

- Efficiency in Data Retrieval: Indexing significantly enhances the efficiency of data retrieval in large datasets.

- Space-Time Trade-off: The benefits of faster queries come at the cost of additional storage space due to the index.

- Applicability: Indexing is particularly useful for large tables and frequent query operations.

- Selective Use: While indexes are beneficial, they should be used selectively based on query patterns and table sizes.

- Maintenance Consideration: Indexes require maintenance, which should be considered in the overall database management strategy.